In [ ]:
import pandas as pd
import os
import json
import numpy as np
from itertools import groupby
import matplotlib.pyplot as plt
from scipy import stats,signal
import matplotlib as mpl
from sklearn.linear_model import LogisticRegression
import random
import re
import csv
from IPython.display import HTML, display, Image
import tabulate
import math as m
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
import statsmodels.formula.api as smf

mpl.rcParams['lines.linewidth'] = 2
mpl.rcParams['axes.titlesize'] = 20
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['lines.markersize'] = 10
mpl.rcParams['xtick.labelsize'] = 20
mpl.rcParams['ytick.labelsize'] = 20
mpl.rcParams['axes.linewidth'] = 1
#mpl.rcParams['xtick.major.size'] = 20
mpl.rcParams['xtick.major.width'] = 1
#mpl.rcParams['xtick.minor.size'] = 10
mpl.rcParams['xtick.minor.width'] = 1
mpl.rcParams['ytick.major.width'] = 1
mpl.rcParams['ytick.minor.width'] = 1
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

fday = [1,2,3,4,5,6,7,8,9,10]
fsession = [1,2]
excluded_miss = [(1011,11), (1011,18), (1014,12)]
excluded_time = [(1008,2),(1009,9)]
excluded = excluded_miss+excluded_time

In [ ]:
current_path = os.path.abspath(os.getcwd())
parent_path = os.path.abspath(os.path.join(current_path, os.pardir))
grand_parent_path = os.path.abspath(os.path.join(parent_path, os.pardir))
main_path = os.path.abspath(os.path.join(grand_parent_path, os.pardir))

path_results = main_path+'/results/gabor/'
path_data = main_path+'/data/jatos_gabor_data/tanda1/'

In [ ]:
def completeWithZero(domain, xlista, ylista):
    if len(domain)==len(xlista):
        return ylista
    elif len(domain)<len(xlista):
        print(domain,xlista)
        return 'Error'
    else:
        set1 = set(domain)
        set2 = set(xlista)
        # missed values
        missing = list(sorted(set1 - set2))
        # index of missed values
        index = []
        [index.append(domain.index(elem)) for elem in missing]
        # insert zero in ylista
        [ylista.insert(elem,0) for elem in index]
        return ylista

In [ ]:
# list of participants 
participants = [991+i for i in range(13)]+[1005+i for i in range(10)]

In [ ]:
StairC = {}
for part in participants:
    StairC[part] = {}

In [ ]:
for day in fday:
    for session in fsession:
        sessionid = 2*day-2+session
        
        path = path_results+'day'+str(day)+'/session'+str(session)+'/'
        # data_files = [f for f in os.listdir(path) if f.endswith('_day'+str(day)+'_session'+str(session))]

        stair_files = [f for f in os.listdir(path) if f.startswith('staircase_')]
        aux_st = [f.replace('staircase_Sub','') for f in stair_files]
        subj_stair = [int(f.replace('_Day'+str(day)+'_Sess'+str(session)+'.json','')) for f in aux_st]
        sorted_subj_stair = sorted(subj_stair)
        index_subj_stair = [subj_stair.index(elem) for elem in sorted_subj_stair]
        sorted_stair_files = [stair_files[i] for i in index_subj_stair]

        ind = -1
        for part in sorted_subj_stair:
            part_ses = str(part)+'_'+str(sessionid)
            if part_ses not in excluded:
                f_stair = sorted_stair_files[ind]
                filename=path+f_stair
                with open(filename) as f_stair:
                    data_stair = json.load(f_stair)
                for k, v in data_stair.items():
                    globals()[k]=v 
                StairC[part][sessionid] = np.array(Cnoise)
                ind += 1

In [ ]:
np.arange(1,20)

In [ ]:
meanStairC,stdStairC = {},{}
for part in participants:
    meanStairC[part] = sum(StairC[part].values())/len(StairC[part])
    stdStairC[part] = np.std(np.array(list(StairC[part].values())),axis=0)

In [ ]:
len(StairC[991])

In [ ]:
ind = -1
fig, ax = plt.subplots(6,4,figsize=(18,22))
plt.subplots_adjust(wspace = 0.2)
plt.subplots_adjust(hspace = 0.6)
for part in participants:

    ind += 1
    ind1 = ind%6
    ind2 = int(round(ind/6,1))
    # ax[ind1,ind2].set_title('participante:'+str(part))
    # ax[ind1,ind2].plot(np.arange(1,61),Cresult,'ok')
    
    # ax[ind1,ind2].fill_between(np.arange(1,61),meanStairC[part]-stdStairC[part], 
    #                             meanStairC[part]+stdStairC[part], alpha=0.2, color = 'k')
    for k in StairC[part].keys():
        ax[ind1,ind2].plot(np.arange(1,61),StairC[part][k],'k', alpha=0.2)
    ax[ind1,ind2].set_ylim(0,1)
    if ind1<5:
        ax[ind1,ind2].set_xticks([0,20,40,60])
        ax[ind1,ind2].set_xticklabels([])        
    if ind2>0:
        ax[ind1,ind2].set_yticks([0,0.5,1])
        ax[ind1,ind2].set_yticklabels([])
    ax[ind1,ind2].plot(np.arange(1,61),meanStairC[part],color=[0.7,0.3,0.1])
    ax[ind1,0].set_ylabel('noise level')
    ax[5,ind2].set_xlabel('trial number')
    ax[4,3].set_xlabel('trial number')
    ax[4,3].set_xticklabels([0,20,40,60])  
    ax[5,3].axis('off')
plt.tight_layout()
plt.savefig('traces_staircase.png')
plt.show()